In [1]:
!pip install fsspec==2025.3.2
!pip install datasets transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 re

In [2]:
from datasets import load_dataset

# Example
ds = load_dataset("tau/commonsense_qa", split="train[:5]")
print(ds)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 5
})


In [3]:
from datasets import load_dataset

ds          = load_dataset("tau/commonsense_qa", split="train[:5]")
questions   = ds["question"]
choices_all = ds["choices"]
answer_keys = ds["answerKey"]   # list of "A", "B", ...

texts = []
for q, choice_dict in zip(questions, choices_all):
    prompt = "Question: " + q + "\n" + "\n".join(
        f"{lbl}. {txt}"
        for lbl, txt in zip(choice_dict["label"], choice_dict["text"])
    )
    texts.append(prompt)

labels = []
for choice_dict, key in zip(choices_all, answer_keys):
    # find the index of the correct label letter in the 'label' list
    idx = choice_dict["label"].index(key)
    # grab that text
    labels.append(choice_dict["text"][idx])

print(texts[0])
print("→ label:", labels[0])

Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
A. ignore
B. enforce
C. authoritarian
D. yell at
E. avoid
→ label: ignore


In [ ]:
%env HF_TOKEN= "Your HF Token here"

In [3]:
import os

HF_TOKEN = os.environ.get("HF_TOKEN")
if HF_TOKEN is None:
    raise ValueError("Please set the HF_TOKEN env var to your Hugging Face token")

In [4]:
!pip install accelerate

In [4]:
!pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Write me a poem about Machine Learning. I’ll be happy to read it. If you want, you can share it with me here.



##commonsense_qa

## shot=1, beam_size=1

In [5]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # just 1‐shot
        retrieve_k: int = 10,   # top‐10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # SBERT retriever
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name, trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,        # truncate to 128
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        # 1) retrieve top‐K by SBERT
        q_emb      = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb  = self.embedder.encode(demos,   convert_to_tensor=True)
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) 1‐shot beam search (beam_size=1)
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, batch_xc, batch_c, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq: continue
                    batch_xc.append(f"{d}\n\n{prompt}\nAnswer:")
                    batch_c .append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xc = self._batch_ce(batch_xc)
            H_c  = self._batch_ce(batch_c)
            for (seq, d, cum), hxc, hc in zip(parents, H_xc, H_c):
                new_beams.append((seq + [d], cum + (hxc - hc)))
            # keep only top‐1
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[:self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        return out.rsplit("Answer:",1)[-1].strip().split()[0]

    def evaluate(self, prompts, labels, choices_list, demos):
        # precompute embeddings once
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Pred: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} examples: {100*acc:.2f}%")

if __name__=="__main__":
    # only 100 examples to fit in memory
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem   = ex["question"]
        lab, txt = ex["choices"]["label"], ex["choices"]["text"]
        p = "Question: " + stem + "\n" + "\n".join(f"{l}. {t}" for l,t in zip(lab, txt))
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    demos = [f"{p}\nAnswer: {g}" for p,g in zip(prompts, labels)]

    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 1,
    )
    model.evaluate(prompts, labels, choices_list, demos)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
Pred: 'fun', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Where would you see some people doing jumping jacks? ↵ A. military base ↵ B. opera ↵ C. water fountain ↵ D. bus stop ↵ E. church ↵ Answer: military base
Pred: 'D', Gold: 'beach'

=== Example 2 ===
Q: Question: What does playing soccer for a long time lead to? ↵ A. excitement ↵ B. fatigue ↵ C. anger ↵ D. hurting ↵ E. getting tired
Choices: ['excitement', 'fatigue', '

### shot= 1, beam_size= 2

In [12]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # just 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # ← changed to 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT retriever
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size  # ← now 2

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,        # truncate to 128
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (now beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep the top-2 sequences
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Given demos + prompt, generate one token after "Answer:" and return it.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        return out.rsplit("Answer:",1)[-1].strip().split()[0]

    def evaluate(self, prompts, labels, choices_list, demos):
        # precompute embeddings once
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} examples: {100*acc:.2f}%")

if __name__ == "__main__":
    # only 100 examples to fit in memory
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = "Question: " + stem + "\n" + "\n".join(f"{l}. {t}" for l,t in zip(lab, txt))
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # demo-pool entries: "<prompt>\nAnswer: <gold>"
    demos = [f"{p}\nAnswer: {g}" for p,g in zip(prompts, labels)]

    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 2,    # ← instantiate with 2
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
Predicted: 'C', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Where would you see some people doing jumping jacks? ↵ A. military base ↵ B. opera ↵ C. water fountain ↵ D. bus stop ↵ E. church ↵ Answer: military base
Predicted: 'D', Gold: 'beach'

=== Example 2 ===
Q: Question: What does playing soccer for a long time lead to? ↵ A. excitement ↵ B. fatigue ↵ C. anger ↵ D. hurting ↵ E. getting tired
Choices: ['excitement', 'fat

## shot= 1, beam_size= 3

In [13]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # ← now 3
        device: str = None,
    ):
        # Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 1) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 2) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size  # ← updated

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=3) using CE diff.
        """
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # Top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # Beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep top-3 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Generate one token after “Answer:”, return it.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        return out.rsplit("Answer:",1)[-1].strip().split()[0]

    def evaluate(self, prompts, labels, choices_list, demos):
        # Precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} examples: {100*acc:.2f}%")

if __name__=="__main__":
    # Only 100 examples to fit in memory
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = "Question: " + stem + "\n" + "\n".join(f"{l}. {t}" for l,t in zip(lab, txt))
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # Demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p,g in zip(prompts, labels)]

    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 3,  # ← use beam_size=3 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
Predicted: 'C', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Where would you see some people doing jumping jacks? ↵ A. military base ↵ B. opera ↵ C. water fountain ↵ D. bus stop ↵ E. church ↵ Answer: military base
Predicted: 'D', Gold: 'beach'

=== Example 2 ===
Q: Question: What does playing soccer for a long time lead to? ↵ A. excitement ↵ B. fatigue ↵ C. anger ↵ D. hurting ↵ E. getting tired
Choices: ['excitement', 'fat

### shot = 2, beam_size= 1

In [8]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Gemma-2B for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # else fallback
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of question strings
        labels:        list of gold answer strings
        choices_list:  list of choice-lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n", " ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} exs: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__ == "__main__":
    # 1) load a small slice
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, choice lists
    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem   = ex["question"]
        lab, txt = ex["choices"]["label"], ex["choices"]["text"]
        p      = "Question: " + stem + "\n" + "\n".join(f"{l}. {t}" for l,t in zip(lab, txt))
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 1,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
 • Question: What could happen to a paper if you leave it outside even if it does not move? ↵ A. one material often recycled ↵ B. ripped ↵ C. saturated with water ↵ D. one dimensional ↵ E. crumpled ↵ Answer: saturated with water
Predicted: 'alive', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Where would you see some people doing jumping jacks? ↵ A. military base ↵ B. opera ↵ C. water fountain ↵ D. bus stop ↵ E. church ↵ A

### shot= 2, beam_size= 2

In [16]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for fast retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        # 1) embed prompt & pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 2) retrieve top-K by cosine
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 3) beam‐search over CE‐diff
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        # fallback: choose choice with min CE(context + choice)
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        # try 1‐token generation
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        # parse after "Answer:"
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # otherwise fallback
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        # pre‐embed pool once
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n", " ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} examples: {100*acc:.2f}%")

if __name__=="__main__":
    # 1) load 100 examples
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, choice‐lists
    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = "Question: " + stem + "\n" + "\n".join(f"{l}. {t}" for l, t in zip(lab, txt))
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # 3) demo‐pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with shot=2, beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 2,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What could happen to a paper if you leave it outside even if it does not move? ↵ A. one material often recycled ↵ B. ripped ↵ C. saturated with water ↵ D. one dimensional ↵ E. crumpled ↵ Answer: saturated with water
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
Predicted: 'alive', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Where would you see some people doing jumping jacks? ↵ A. military base ↵ B. opera ↵ C. water fountain ↵ D. bus stop ↵ E. church ↵ A

### shot= 2, beam_size= 3

In [17]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam size 3
        device: str = None,
    ):
        # Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """Compute total cross-entropy for each text in `texts`, in batches."""
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """First SBERT‐retrieve top‐K, then 2‐shot beam search (beam_size=3) via CE difference."""
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) retrieve
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, score in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, score))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, score), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], score + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """Fallback: pick the choice with lowest CE(context + ' ' + choice)."""
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Attempt 1-token generation after "Answer:"; if it produces nothing,
        fall back to pure CE-based ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            # take first token after last "Answer:"
            return parts[1].strip().split()[0]
        # fallback if generation failed
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """Run beam search + classification over the provided examples."""
        # pre‐embed the pool
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n", " ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} examples: {100*acc:.2f}%")

if __name__=="__main__":
    # 1) load a small slice
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice‐lists
    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = "Question: " + stem + "\n" + "\n".join(f"{l}. {t}" for l, t in zip(lab, txt))
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # 3) demo‐pool entries: "<prompt>\nAnswer: <gold>"
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with shot=2 and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 3,  # now using beam_size=3
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
 • Question: A person lived in my what? ↵ A. demonstration ↵ B. conference ↵ C. building ↵ D. garage ↵ E. mouth ↵ Answer: garage
Predicted: 'fun', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Where is a bald eagle safe? ↵ A. pine tree ↵ B. open country ↵ C. in washington ↵ D. wildlife refuge ↵ E. sky ↵ Answer: wildlife refuge
 • Question: Where would you see some people doing jumping jacks? ↵ A. military base ↵ B. opera ↵ 

### shot= 3 beam_size = 1

In [9]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Gemma-2B for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of question strings
        labels:        list of gold answer strings
        choices_list:  list of choice-lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n", " ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} exs: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__ == "__main__":
    # 1) load a small slice
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, choice lists
    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = (
            "Question: " + stem + "\n"
            + "\n".join(f"{l}. {t}" for l, t in zip(lab, txt))
        )
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # 3) demo‐pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 1,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
 • Question: What could happen to a paper if you leave it outside even if it does not move? ↵ A. one material often recycled ↵ B. ripped ↵ C. saturated with water ↵ D. one dimensional ↵ E. crumpled ↵ Answer: saturated with water
 • Question: A person lived in my what? ↵ A. demonstration ↵ B. conference ↵ C. building ↵ D. garage ↵ E. mouth ↵ Answer: garage
Predicted: 'garage', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Wh

### shot= 3, beam_size= 2

In [11]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # now 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam of size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Gemma-2B for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of question strings
        labels:        list of gold answer strings
        choices_list:  list of choice-lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n", " ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} exs: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__ == "__main__":
    # 1) Load a small slice for speed
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    # 2) Build prompts, gold labels, and choice lists
    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = (
            "Question: " + stem + "\n"
            + "\n".join(f"{l}. {t}" for l, t in zip(lab, txt))
        )
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # 3) Demo‐pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) Run with 3-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 2,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What could happen to a paper if you leave it outside even if it does not move? ↵ A. one material often recycled ↵ B. ripped ↵ C. saturated with water ↵ D. one dimensional ↵ E. crumpled ↵ Answer: saturated with water
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
 • Question: A person lived in my what? ↵ A. demonstration ↵ B. conference ↵ C. building ↵ D. garage ↵ E. mouth ↵ Answer: garage
Predicted: 'garage', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: Wh

### shot= 3, beam_size = 3

In [10]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam of size 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Gemma-2B for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of question strings
        labels:        list of gold answer strings
        choices_list:  list of choice-lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Q:", p.replace("\n", " ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n", " ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ CommonsenseQA Accuracy on {len(prompts)} exs: {100*acc:.2f}% ({correct}/{len(prompts)})")


if __name__ == "__main__":
    # 1) load a small slice
    ds = load_dataset("tau/commonsense_qa", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, choice lists
    prompts, labels, choices_list = [], [], []
    for ex in ds:
        stem, lab, txt = ex["question"], ex["choices"]["label"], ex["choices"]["text"]
        p = (
            "Question: " + stem + "\n"
            + "\n".join(f"{l}. {t}" for l, t in zip(lab, txt))
        )
        prompts.append(p)
        labels.append(txt[lab.index(ex["answerKey"])])
        choices_list.append(txt)

    # 3) demo‐pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 3,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Q: Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black
Choices: ['alive', 'health', 'fun', 'playing dead', 'black']
Demos:
 • Question: What is a grumpy person likely to have? ↵ A. headache ↵ B. understand themselves ↵ C. compliments ↵ D. own house ↵ E. happiness ↵ Answer: headache
 • Question: A person lived in my what? ↵ A. demonstration ↵ B. conference ↵ C. building ↵ D. garage ↵ E. mouth ↵ Answer: garage
 • Question: When you play around with your dog they will have? ↵ A. alive ↵ B. health ↵ C. fun ↵ D. playing dead ↵ E. black ↵ Answer: fun
Predicted: 'fun', Gold: 'fun'

=== Example 1 ===
Q: Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. beach ↵ E. beans
Choices: ['in the basement', 'west', 'solar system', 'beach', 'beans']
Demos:
 • Question: What is a great place to lay in the sun? ↵ A. in the basement ↵ B. west ↵ C. solar system ↵ D. 

## AG News

### shot= 1, beam_size= 1

In [8]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # now 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 1-shot and beam_size=1
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 1,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
Predicted: 'Business', Gold: 'Sci/Tech'

=== Example 1 ===
Article prompt: News article: ↵ Google IPO: Type in 'confusing,' 'secrecy' I've submitted my bid to buy shares of Google Inc. in the computer search company's giant auction-style initial public offering. That could turn out to be the good news or the bad news. ↵  ↵ Category? (World / S

### shot= 1, beam_size= 2

In [13]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam of size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search (single step, since shot=1)
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-2
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 1-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 2,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
Predicted: 'Business', Gold: 'Sci/Tech'

=== Example 1 ===
Article prompt: News article: ↵ Google IPO: Type in 'confusing,' 'secrecy' I've submitted my bid to buy shares of Google Inc. in the computer search company's giant auction-style initial public offering. That could turn out to be the good news or the bad news. ↵  ↵ Category? (World / S

### shot= 1, beam_size= 3

In [12]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam of size 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search (single step, since shot=1)
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-3
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 1-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 3,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
Predicted: 'Business', Gold: 'Sci/Tech'

=== Example 1 ===
Article prompt: News article: ↵ Google IPO: Type in 'confusing,' 'secrecy' I've submitted my bid to buy shares of Google Inc. in the computer search company's giant auction-style initial public offering. That could turn out to be the good news or the bad news. ↵  ↵ Category? (World / S

### shot= 2, beam_size= 1

In [9]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # now 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot and beam_size=1
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 1,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
Predicted: 'Business', Gold: 'Sci/Tech'

=== Example 1 ===
Article prompt: News article: ↵ Google IPO: Type 

### shot= 2, beam_size= 2

In [10]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam of size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 2,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Quality Gets Swept Away Quality Distribution is hammered after reporting a large loss for the second quarter. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
Predicted: 'Business', Gold: 'Sci/Tech'

=== Example 1 ===
Article prompt: News article: ↵ Google IPO: Type in 'confusing,' 'secrecy' I've subm

### shot=2, beam_size= 3

In [11]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam of size 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-3 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 3,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Quality Gets Swept Away Quality Distribution is hammered after reporting a large loss for the second quarter. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
Predicted: 'Business', Gold: 'Sci/Tech'

=== Example 1 ===
Article prompt: News article: ↵ Google IPO: Type in 'c

### shot= 3, beam_size= 1

In [7]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        # same four for every example
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=1
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 1,  # ← beam_size=1 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Quality Gets Swept Away Quality Distribution is hammered after reporting a large loss for

### shot= 3, beam_size= 2

In [6]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam of size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        # same four for every example
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 2,  # ← beam_size=2 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about to get far creepier ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Quality Gets Swept Away Quality Distribution is hammered after reporting a large loss for the second quarter. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are

### shot= 3, beam_size= 3

In [5]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam of size 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of news‐article prompts
        labels:        list of gold category names
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Article prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ AG-News Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of AG-News
    ds = load_dataset("ag_news", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["World", "Sports", "Business", "Sci/Tech"]
    for ex in ds:
        prompt = (
            f"News article:\n{ex['text']}\n\n"
            "Category? (World / Sports / Business / Sci/Tech)"
        )
        prompts.append(prompt)
        labels.append(fixed_choices[ex["label"]])
        # same four for every example
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 3,
    )
    model.evaluate(prompts, labels, choices_list, demos)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]


=== Example 0 ===
Article prompt: News article: ↵ Missing June Deals Slow to Return for Software Cos. (Reuters) Reuters - The mystery of what went wrong for the\software industry in late June when sales stalled at more than\20 brand-name companies is not even close to being solved\although the third quarter is nearly halfway over. ↵  ↵ Category? (World / Sports / Business / Sci/Tech)
Choices: ['World', 'Sports', 'Business', 'Sci/Tech']
Demos:
 • News article: ↵ Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ Quality Gets Swept Away Quality Distribution is hammered after reporting a large loss for the second quarter. ↵  ↵ Category? (World / Sports / Business / Sci/Tech) ↵ Answer: Business
 • News article: ↵ The Age of Doom In 1993 six geeks had a digital nightmare that changed the culture. It's about

## SST 5

### shot= 1, beam_size= 1

In [6]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # ← now 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search (single step, since shot=1)
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-1 beam
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 1-shot and beam_size=1
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 1,  # ← beam_size=1 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
Predicted: 'Very', Gold: 'Positive'

=== Example 1 ===
Review prompt: Review: analyze that regurgitates and waters down many of the previous film 's successes , with a few new swings thrown in . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:


### shot= 1, beam_size= 2

In [5]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam of size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search (single step, since shot=1)
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-2 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 1-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 2,
    )
    model.evaluate(prompts, labels, choices_list, demos)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/171k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
Predicted: 'Positive', Gold: 'Positive'

=== Example 1 ===
Review prompt: Review: analyze that regurgitates and waters down many of the previous film 's successes , with a few new swings thrown in . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Dem

### shot= 1, beam_size= 3

In [7]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 1,          # 1-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # ← now 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size  # ← updated

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 1-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search (single step, since shot=1)
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-3 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 1-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 1,
        retrieve_k   = 10,
        beam_size    = 3,  # ← beam_size=3 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
Predicted: 'Positive', Gold: 'Positive'

=== Example 1 ===
Review prompt: Review: analyze that regurgitates and waters down many of the previous film 's successes , with a few new swings thrown in . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Dem

### shot= 2, beam_size= 1

In [12]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # beam of size 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama-3.2-3B-Instruct for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=1) using CE diff.
        """
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-1 beam
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot and beam_size=1
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,    # ← shot=2 here
        retrieve_k   = 10,
        beam_size    = 1,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
 • Review: this is a very ambitious project for a fairly inexperienced filmmaker , but good actors , good poetry and good music help sustain it . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Positive
Predicted: 'Positive', Gold: 'Positive'

=== Example 1 ===
Review prompt: Review: analyze that regurgitates and wate

### shot= 2, beam_size= 2

In [13]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # beam of size 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama-3.2-3B-Instruct for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=2) using CE diff.
        """
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-2 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        # return the best sequence from the beam
        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 2,  # ← beam_size=2 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
 • Review: this is a very ambitious project for a fairly inexperienced filmmaker , but good actors , good poetry and good music help sustain it . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Positive
Predicted: 'Positive', Gold: 'Positive'

=== Example 1 ===
Review prompt: Review: analyze that regurgitates and wate

### shot= 2, beam_size= 3

In [8]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 2,          # ← now 2-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam of size 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama-3.2-3B-Instruct for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 2-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-3 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 2-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 2,
        retrieve_k   = 10,
        beam_size    = 3,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Positive
 • Review: this is a very ambitious project for a fairly inexperienced filmmaker , but good actors , good poetry and good music help sustain it . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Positive
Predicted: 'Positive', Gold: 'Positive'

=== Example 1 ===
Review prompt: Review: analyze that reg

### shot= 3, beam_size= 1

In [10]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 1,     # ← now beam_size = 1
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama-3.2-3B-Instruct for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=1) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-1 beam
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=1
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 1,  # ← beam_size=1 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
 • Review: this is a very ambitious project for a fairly inexperienced filmmaker , but good actors , good poetry and good music help sustain it . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Positive
 • Review: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films ↵

### shot= 3, beam_size= 2

In [11]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 2,     # ← now beam_size = 2
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama-3.2-3B-Instruct for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=2) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-2 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=2
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 2,  # ← beam_size=2 here
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Negative
 • Review: this is a very ambitious project for a fairly inexperienced filmmaker , but good actors , good poetry and good music help sustain it . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Positive
 • Review: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films ↵

### shot= 3, beam_size= 3

In [9]:
# ── Install these once in your environment: ────────────────────────────────────
# pip install transformers datasets sentence-transformers accelerate

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

class Se2ICLQA:
    def __init__(
        self,
        model_name: str = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model: str = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot: int = 3,          # ← now 3-shot
        retrieve_k: int = 10,   # top-10 SBERT
        beam_size: int = 3,     # beam of size 3
        device: str = None,
    ):
        # 1) Device
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # 2) SBERT for retrieval
        self.embedder = SentenceTransformer(embed_model, device=self.device)

        # 3) Llama-3.2-3B-Instruct for CE scoring & generation
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tok.pad_token is None:
            self.tok.pad_token = self.tok.eos_token
        self.lm = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.lm.eval()

        self.shot       = shot
        self.retrieve_k = retrieve_k
        self.beam_size  = beam_size

    @torch.inference_mode()
    def _batch_ce(self, texts, batch_size=8):
        """
        Compute total cross-entropy for each text in `texts`, in batches.
        """
        ces = []
        for i in range(0, len(texts), batch_size):
            chunk = texts[i : i+batch_size]
            enc = self.tok(
                chunk,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128,
            ).to(self.device)
            out = self.lm(enc.input_ids, labels=enc.input_ids)
            lengths = enc.attention_mask.sum(dim=1).float()
            ces.extend((out.loss * lengths).cpu().tolist())
        return ces

    def beam_search(self, prompt: str, demos: list[str]):
        """
        1) Retrieve top-K demos by SBERT.
        2) Perform 3-shot beam search (beam_size=3) using CE diff.
        """
        # embed prompt + demo pool
        q_emb     = self.embedder.encode([prompt], convert_to_tensor=True)
        demos_emb = self.embedder.encode(demos,   convert_to_tensor=True)

        # 1) top-K retrieval
        sims, idxs = (demos_emb @ q_emb.T).squeeze(1).topk(self.retrieve_k)
        candidates = [demos[i] for i in idxs.cpu().tolist()]

        # 2) beam search
        beams = [([], 0.0)]
        for _ in range(self.shot):
            new_beams, xb, cb, parents = [], [], [], []
            for seq, cum in beams:
                for d in candidates:
                    if d in seq:
                        continue
                    xb.append(f"{d}\n\n{prompt}\nAnswer:")
                    cb.append(f"{d}\n\nAnswer:")
                    parents.append((seq, d, cum))
            H_xb = self._batch_ce(xb)
            H_cb = self._batch_ce(cb)
            for (seq, d, cum), hxb, hcb in zip(parents, H_xb, H_cb):
                new_beams.append((seq + [d], cum + (hxb - hcb)))
            # keep only top-3 beams
            new_beams.sort(key=lambda x: x[1], reverse=True)
            beams = new_beams[: self.beam_size]

        return beams[0][0]

    def classify_by_ce(self, context: str, choices: list[str]) -> str:
        """
        Fallback: score each choice by CE(context + " " + choice) and pick lowest.
        """
        texts = [f"{context} {c}" for c in choices]
        ces   = self._batch_ce(texts)
        return choices[int(np.argmin(ces))]

    def classify(self, prompt: str, demos: list[str], choices: list[str]) -> str:
        """
        Try 1-token generation; if it fails, fallback to CE ranking.
        """
        ctx = "\n\n".join(demos) + f"\n\n{prompt}\nAnswer:"
        enc = self.tok(
            ctx,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(self.device)
        gen = self.lm.generate(
            **enc,
            max_new_tokens=1,
            pad_token_id=self.tok.eos_token_id
        )
        out = self.tok.decode(gen[0], skip_special_tokens=True)
        parts = out.rsplit("Answer:", 1)
        if len(parts) == 2 and parts[1].strip():
            return parts[1].strip().split()[0]
        # fallback if no token generated
        return self.classify_by_ce(ctx, choices)

    def evaluate(self, prompts, labels, choices_list, demos):
        """
        prompts:       list of review prompts
        labels:        list of gold sentiment labels
        choices_list:  list of choice‐lists per example
        demos:         list of "<prompt>\nAnswer: <gold>" for pool
        """
        # precompute demo embeddings
        self.demos     = demos
        self.demos_emb = self.embedder.encode(demos, convert_to_tensor=True)

        correct = 0
        for i, (p, gold, choices) in enumerate(zip(prompts, labels, choices_list)):
            sel  = self.beam_search(p, demos)
            pred = self.classify(p, sel, choices)
            if pred == gold:
                correct += 1

            if i < 3:
                print(f"\n=== Example {i} ===")
                print("Review prompt:", p.replace("\n"," ↵ "))
                print("Choices:", choices)
                print("Demos:")
                for d in sel:
                    print(" •", d.replace("\n"," ↵ "))
                print(f"Predicted: {pred!r}, Gold: {gold!r}")

        acc = correct / len(prompts)
        print(f"\n⇒ SST-5 Accuracy ({len(prompts)} exs): {100*acc:.2f}% ({correct}/{len(prompts)})")

if __name__=="__main__":
    # 1) load a small slice of SST-5
    ds = load_dataset("SetFit/sst5", split="train[:100]").shuffle(seed=0)

    # 2) build prompts, labels, and choice lists
    prompts, labels, choices_list = [], [], []
    fixed_choices = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
    label_map = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
    }
    for ex in ds:
        prompt = (
            f"Review: {ex['text']}\n\n"
            "Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)"
        )
        prompts.append(prompt)
        labels.append(label_map[ex["label"]])
        choices_list.append(fixed_choices)

    # 3) demo-pool entries
    demos = [f"{p}\nAnswer: {g}" for p, g in zip(prompts, labels)]

    # 4) run with 3-shot and beam_size=3
    model = Se2ICLQA(
        model_name   = "meta-llama/Llama-3.2-3B-Instruct",
        embed_model  = "sentence-transformers/paraphrase-mpnet-base-v2",
        shot         = 3,
        retrieve_k   = 10,
        beam_size    = 3,
    )
    model.evaluate(prompts, labels, choices_list, demos)

Repo card metadata block was not found. Setting CardData to empty.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== Example 0 ===
Review prompt: Review: the wonderfully lush morvern callar is pure punk existentialism , and ms. ramsay and her co-writer , liana dognini , have dramatized the alan warner novel , which itself felt like an answer to irvine welsh 's book trainspotting . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive)
Choices: ['Very Negative', 'Negative', 'Neutral', 'Positive', 'Very Positive']
Demos:
 • Review: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Very Positive
 • Review: this is a very ambitious project for a fairly inexperienced filmmaker , but good actors , good poetry and good music help sustain it . ↵  ↵ Sentiment? (Very Negative / Negative / Neutral / Positive / Very Positive) ↵ Answer: Positive
 • Review: becomes a bit of a mishmash : a tearjerker that does n't and a thriller that wo n't . ↵